In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import json
import os
import pandas as pd
from utils.misc import get_summary, get_query

os.chdir('/home/s2310409/workspace/coliee-2024/')



In [2]:
device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", device_map='auto')
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
def load_data(dir):
    with open(dir, 'r') as fp:
        train_data = json.load(fp)

    data = []
    for key in train_data.keys():
        data.append([key, train_data[key]])

    return pd.DataFrame(data, columns=['source', 'target'])

data_df = load_data(f'dataset/test.json')
data_df

,source,target
0,070318.txt,[015076.txt]
1,077960.txt,"[009054.txt, 040860.txt]"
2,042319.txt,"[093691.txt, 075956.txt, 084953.txt, 022987.txt]"
3,041766.txt,[039269.txt]
4,077407.txt,[038669.txt]
...,...,...
314,085079.txt,"[044669.txt, 003144.txt]"
315,031370.txt,"[096341.txt, 060602.txt, 047107.txt, 084522.tx..."
316,085828.txt,"[004301.txt, 074887.txt, 088994.txt]"
317,024957.txt,"[015009.txt, 080348.txt]"


In [11]:
with open('dataset/c2023/bm25_candidates_test.json', 'r') as fp:
    candidate_dict = json.load(fp)

In [29]:

# messages = [
#     {"role": "user", "content": "What is your favourite condiment?"},
#     {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
#     {"role": "user", "content": "Do you have mayonnaise recipes?"}
# ]

def reranking_prompting(list_articles, query_content):
    prompting = f"In bellow articles:  "
    for a_id in list_articles:
        a_content = get_summary(a_id)
        prompting = prompting + f"\n##Article {a_id}: {a_content},"
        
    prompting = prompting +f"\n##Question: which articles really relevant to the following article? Answer the article name only. \n##Article: {query_content}"
    prompting = prompting + "\n##Answer:"
    return prompting

source = data_df['source'][0]
candidates = candidate_dict[source][0:7]
text = reranking_prompting(candidates, get_summary(source))

encodeds = tokenizer(text, return_tensors="pt")
model_inputs = encodeds.to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [30]:
print(candidates)
print(decoded[0].replace(text, ''))

['032432.txt', '071237.txt', '019716.txt', '027423.txt', '012462.txt', '028967.txt', '030534.txt']
<s>  Article 032432.txt, Article 071237.txt, Article 019716.txt, Article 027423.txt and Article 012462.txt.
In each of the articles mentioned above, issues of bias or fairness have been raised. Specifically, in Article 032432.txt, the applicant argues that the officer responsible for the second PRRA application was biased in the processing of the first. In Article 071237.txt, the applicant claims that the Board intervened in an intimidating and intrusive manner during the hearing, and that a member of the Board may have used information obtained by his own means to question the applicant. In Article 019716.txt, the applicants argue that the Commissioner did not consider all relevant information and may have been biased based on its earlier decision. In Article 027423.txt, the applicant argues that the Commissioner had a clear and obvious reasonable apprehension of bias due to his past inv